# Data Processing

In [1]:
#!/usr/bin/env python3
import time
from multiprocessing import cpu_count
from typing import Union, NamedTuple

from torch.utils import data
from torch import Tensor
from typing import Tuple
import torch
import torch.backends.cudnn
import numpy as np
from torch import nn, optim
from torch.nn import functional as F
import torchvision.datasets
from torch.optim.optimizer import Optimizer
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from torch.utils.data import Dataset

import argparse
from pathlib import Path

from PIL import Image
from torchvision import transforms

torch.backends.cudnn.benchmark = True


def crop_to_region(coords: Tuple[int], img: Tensor, crop_size: int=42) -> Tensor:
    """ 
    Given coordinates in the form Tuple[int](y, x), return a cropped
    sample of the input imaged centred at (y, x), matching the input size.
    Args:
        coords (Tuple[int]): The input coordinates (y, x) where the crop will be
        centred.
        img (Tensor): The input image, either 3x400x400, 3x250x250, 3x150x150
        crop_size (int, optional): The size of the returned crop. Defaults to 42.

    Returns:
        Tensor: The image cropped with central coordinates at (y, x) of size 
        (3 x size x size) # is size here referring to 42?
    """
    _, H, W = img.shape
    y, x = coords
    y_min, x_min = max(0, y-crop_size//2), max(0, x-crop_size//2)
    y_max, x_max = min(H, y+crop_size//2), min(W, x+crop_size//2)
    region = img[:, y_min:y_max, x_min:x_max]
    if region.shape[1] < crop_size:
        to_pad = crop_size - region.shape[1]
        padding = (0, 0, to_pad, 0) if (y-crop_size//2) < 0 else (0, 0, 0, to_pad)
        region = F.pad(region, padding, mode='replicate')

    if region.shape[2] < crop_size:
        to_pad = crop_size - region.shape[2]
        padding = (to_pad, 0, 0, 0) if (x-crop_size//2) < 0 else (0, to_pad, 0, 0)
        region = F.pad(region, padding, mode='replicate')
    return region

class MIT(data.Dataset):
    def __init__(self, dataset_path: str):
        """
        Given the dataset path, create the MIT dataset. Creates the
        variable self.dataset which is a list of dictionaries with three keys:
            1) X: For train the crop of image. This is of shape [3, 3, 42, 42]. The 
                first dim represents the crop across each different scale
                (400x400, 250x250, 150x150), the second dim is the colour
                channels C, followed by H and W (42x42). For inference, this is 
                the full size image of shape [3, H, W].
            2) y: The label for the crop. 1 = a fixation point, 0 = a
                non-fixation point. -1 = Unlabelled i.e. val and test
            3) file: The file name the crops were extracted from.
            
        If the dataset belongs to val or test, there are 4 additional keys:
            1) X_400: The image resized to 400x400
            2) X_250: The image resized to 250x250
            3) X_150: The image resized to 150x150
            4) spatial_coords: The centre coordinates of all 50x50 (2500) crops
            
        These additional keys help to load the different scales within the
        dataloader itself in a timely manner. Precomputing all crops requires too
        much storage for the lab machines, and resizing/cropping on the fly
        slows down the dataloader, so this is a happy balance.
        Args:
            dataset_path (str): Path to train/val/test.pth.tar
        """
        self.dataset = torch.load(dataset_path, weights_only=True)
        self.mode = 'train' if 'train' in dataset_path else 'inference'
        self.num_crops = 2500 if self.mode == 'inference' else 1

    def __getitem__(self, index) -> Tuple[Tensor, int]:
        """
        Given the index from the DataLoader, return the image crop(s) and label
        Args:
            index (int): the dataset index provided by the PyTorch DataLoader.
        Returns:
            Tuple[Tensor, int]: A two-element tuple consisting of: 
                1) img (Tensor): The image crop of shape [3, 3, 42, 42]. The 
                first dim represents the crop across each different scale
                (400x400, 250x250, 150x150), the second dim is the colour
                channels C, followed by H and W (42x42).
                2) label (int): The label for this crop. 1 = a fixation point, 
                0 = a non-fixation point. -1 = Unlabelled i.e. val and test.
        """
        sample_index = index // self.num_crops
        # print(sample_index)
        
        img = self.dataset[sample_index]['X']
        
        # Inference crops are not precomputed due to file size, do here instead
        if self.mode == 'inference': 
            _, H, W = img.shape
            crop_index = index % self.num_crops
            crop_y, crop_x = self.dataset[sample_index]['spatial_coords'][crop_index]
            scales = []
            for size in ['X_400', 'X_250', 'X_150']:
                scaled_img = self.dataset[sample_index][size]
                y_ratio, x_ratio = scaled_img.shape[1] / H, scaled_img.shape[2] / W
                
                # Need to rescale the crops central coordinate.
                scaled_coords = (int(y_ratio * crop_y), int(x_ratio * crop_x))
                crops = crop_to_region(scaled_coords, scaled_img)
                scales.append(crops)
            img = torch.stack(scales, axis=1)
            
        label = self.dataset[sample_index]['y']

        return img, label

    def __len__(self):
        """
        Returns the length of the dataset (length of the list of dictionaries * number
        of crops). 
        __len()__ always needs to be defined so that the DataLoader
            can create the batches
        Returns:
            len(self.dataset) (int): the length of the list of dictionaries * number of
            crops.
        """
        return len(self.dataset) * self.num_crops


trainingdata = MIT("data/train_data.pth.tar")
valdata = MIT("data/val_data.pth.tar")
testingdata = MIT("data/test_data.pth.tar")


class NormalisedDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


# each element in self.dataset dictionary which has three components so just get X and y component (so X component 3x3x42x42 and y is label) -> inputs to the CNN


# print(trainingdata.dataset[0]['y'])
# print(trainingdata.dataset[0]['X'])


# Data Augmentation

In [2]:
import numpy as np
import statistics

training_data_augmented = trainingdata.dataset * 2
for i in range(len(trainingdata.dataset), len(training_data_augmented)):
    training_data_augmented[i]['X'] = torch.flip(training_data_augmented[i]['X'], dims=[3])
    
print('done')

red_channel = ([sample['X'][:, 0, :, :] for sample in training_data_augmented])
blue_channel = ([sample['X'][:, 1, :, :] for sample in training_data_augmented])
green_channel = ([sample['X'][:, 2, :, :] for sample in training_data_augmented])

normalised_training_data = training_data_augmented

# red channel is a list of 3x42x42 i.e each resolution 42x42 image only the red channel, likewise with the blue and green channel
# so want the mean of the red channels, blue and green channel

def channel_mean(x):
    return (torch.stack(x).mean().item())

def channel_std(x):
    return (torch.stack(x).std().item())

# def channel_mean(x):
#     channel_sum = 0 # sum is 0 to begin with
#     for i in range(0, len(x)): # go through each item in the list
#         item = x[i] # extract 3x42x42 list
#         for j in range(0, 3): # find the sum of each element of the 3x42x42 list and increment to the running sum
#             for k in range(0, 42):
#                 for l in range(0,42):
#                     channel_sum += item[j][k][l].item()
                    
#     return (channel_sum / (len(x)*3*42*42)) # mean of x i.e. total of all values / number of values 

red_mean = channel_mean(red_channel)
blue_mean = channel_mean(blue_channel)
green_mean = channel_mean(green_channel)

red_std = channel_std(red_channel)
blue_std = channel_std(blue_channel)
green_std = channel_std(green_channel)

print(red_mean, blue_mean, green_mean, red_std, blue_std, green_std)

for i in range(0, len(normalised_training_data)): # goes through the current list
    training_data_X = normalised_training_data[i]['X'] # gets the 3x3x42x42 X
    training_data_X[:,0,:,:] = (training_data_X[:,0,:,:] - red_mean) / red_std # gets red channel values and normalises them
    training_data_X[:, 1, :,:] = (training_data_X[:,1,:,:] - blue_mean) / blue_std
    training_data_X[:, 2, :,:] = (training_data_X[:,2,:,:] - green_mean) / green_std
    
    normalised_training_data[i]['X'] = training_data_X
    #print('in loop')

final_data = trainingdata
final_data.dataset = normalised_training_data

print('hello')

done
0.01992413029074669 -0.003527479013428092 -0.011873003095388412 0.9579979777336121 0.948402464389801 0.9503791928291321
hello


# Format Files and Generate Ground Truth

In [3]:
# this code generates the ground truth dictionary for inference datasets indexed by file name (formatted so they end with _fixMap.jpg)
# ensure the predictions dictionary matches this format for file name

def format_file_name_fixMap_jpg(filename):
    return f"{filename[:-5]}_fixMap.jpg"

def get_filenames_fixMap_jpg(dataset: Dataset):
    filenames = [format_file_name_fixMap_jpg(sample['file']) for sample in dataset]

    return filenames

def get_ground_truth_dict(dataset: Dataset):

    filenames = get_filenames_fixMap_jpg(dataset)

    ground_truth_dict = {}
    transform = transforms.ToTensor()

    for filename in filenames:
        ground_truth_dict[filename] = transform(Image.open(f"data/ALLFIXATIONMAPS/ALLFIXATIONMAPS/{filename}")).squeeze() * 255
    
    return ground_truth_dict

# to show usage
# print(list(get_ground_truth_dict(get_filenames_fixMap_jpg(valdata.dataset)).values())[7].shape)

val_ground_truth_dict = get_ground_truth_dict(valdata.dataset)
test_ground_truth_dict = get_ground_truth_dict(testingdata.dataset)

print(list(get_ground_truth_dict(valdata.dataset).values())[0].shape)
print(valdata.dataset[0]['X'].shape)

torch.Size([768, 1024])
torch.Size([3, 768, 1024])


# Saliency Map Function

In [4]:
import matplotlib.pyplot as plt

def visualise_sal_map(sal_map):

    # Convert the tensor to a numpy array (matplotlib works well with numpy arrays)
    map_np = sal_map.numpy()

    # Plot the tensor as an image
    plt.imshow(map_np, cmap='gray', interpolation='nearest')
    # plt.colorbar()  # Optional: add a colorbar to visualize the range of values
    plt.title('50x50 Saliency Map as Grayscale Image')
    plt.axis('off')  # Turn off axis ticks
    plt.show()

# Define ROC-AUC Evaluation

In [5]:
# metrics code


import numpy as np
from scipy.integrate import simpson


def roc_auc(pred, target, n_points=20, include_prior=False):
        """
        Calculates the Reciever-Operating-Characteristic (ROC) area under
        the curve (AUC) by numerical integration.
        """

        target = np.array(target)/255
                
        # generated = pred
        # changed above comment line to below one as above was throwing error
        generated = np.array(pred)
        # min max normalisation
        generated = (generated - generated.min())/(generated.max() - generated.min())

        def roc(p=0.1):
            x = generated.reshape(-1) > p
            t = target.reshape(-1) > p

            return np.sum(x==t)/len(t)

        calculate_roc = np.vectorize(roc)

        x = np.linspace(0, 1, n_points)
        auc = simpson(calculate_roc(x))/n_points

        return auc

def calculate_auc(preds, targets):
	"""
	inputs -- 2 dictionary with prediction and target images. The 2 dictionaries have the  same number of keys, where each key identifies a unique image. 
	The predictions have the predicted fixation maps while the targets have the ground truth fixation maps which are available from "https://people.csail.mit.edu/tjudd/WherePeopleLook/" 
	"""
	assert preds.keys() == targets.keys()
	mean_auc = 0
	for key in preds.keys():
		mean_auc += roc_auc(preds[key], targets[key])
	mean_auc /= len(preds.keys())
	return mean_auc


# Initialise and Train Mr-CNN

### To see logs:
module load anaconda \
pip install tensorboard \
python3 -m tensorboard.main --logdir=./logs \

In [ ]:
parser = argparse.ArgumentParser(
    description="Train a simple CNN on CIFAR-10",
    formatter_class=argparse.ArgumentDefaultsHelpFormatter,
)
default_dataset_dir = Path.home() / ".cache" / "torch" / "datasets"
parser.add_argument("--dataset-root", default=default_dataset_dir)
parser.add_argument("--log-dir", default=Path("logs"), type=Path)
parser.add_argument("--learning-rate", default=2e-4, type=float, help="Learning rate")
parser.add_argument(
    "--batch-size",
    default=128,
    type=int,
    help="Number of images within each mini-batch",
)
parser.add_argument(
    "--epochs",
    default=20,
    type=int,
    help="Number of epochs (passes through the entire dataset) to train for",
)
parser.add_argument(
    "--val-frequency",
    default=2,
    type=int,
    help="How frequently to test the model on the validation set in number of epochs",
)
parser.add_argument(
    "--log-frequency",
    default=10,
    type=int,
    help="How frequently to save logs to tensorboard in number of steps",
)
parser.add_argument(
    "--print-frequency",
    default=10,
    type=int,
    help="How frequently to print progress to the command line in number of steps",
)
parser.add_argument(
    "-j",
    "--worker-count",
    default=0, #cpu_count(), #note make this 0 if not using lab machine
    type=int,
    help="Number of worker processes used to load data.",
)
parser.add_argument("--sgd-momentum", default=0, type=float)
parser.add_argument("--data-aug-hflip", action="store_true")



class ImageShape(NamedTuple):
    height: int
    width: int
    channels: int


if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")


def main(args):
    # # Stack X tensors and extract y values
    # X_train = torch.stack([sample['X'] for sample in trainingdata.dataset])
    # y_train = torch.tensor([sample['y'] for sample in trainingdata.dataset])

    # # Calculate mean and std for normalization
    # mean_train = X_train.view(X_train.size(0), X_train.size(1), -1).mean(dim=(0, 2))
    # std_train = X_train.view(X_train.size(0), X_train.size(1), -1).std(dim=(0, 2))

    # # Normalize X_train
    # normalised_X_train = (X_train - mean_train[None, :, None, None]) / std_train[None, :, None, None]

    # # Create the custom dataset
    # normalised_trainingdata = NormalisedDataset(normalised_X_train, y_train)

    # Pass the dataset into DataLoader
    # train_loader = DataLoader(normalised_trainingdata, batch_size=128, shuffle=True)

    train_loader = torch.utils.data.DataLoader(
        final_data, #trainingdata,#normalised_trainingdata,
        shuffle=True,
        batch_size=args.batch_size,
        pin_memory=True,
        num_workers=args.worker_count,
    )

    ######################
    
    print(args)
    
    args.dataset_root.mkdir(parents=True, exist_ok=True)
    

    # if False:
    #     train_dataset = None #torchvision.datasets.CIFAR10(args.dataset_root, train=True, download=True, transform=transforms.Compose([transforms.RandomHorizontalFlip(), transforms.ToTensor()]))
    # else:
    #     train_dataset = trainingdata #normalised_trainingdata # torchvision.datasets.CIFAR10(args.dataset_root, train=True, download=True, transform=transforms.ToTensor())

    test_dataset = testingdata.dataset #torchvision.datasets.CIFAR10(args.dataset_root, train=False, download=False, transform=transforms.ToTensor())
    
    
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        shuffle=False,
        batch_size=args.batch_size,
        num_workers=args.worker_count,
        pin_memory=True,
    )

    model = CNN(height=42, width=42, channels=3, class_count=2)

    # for torchviz visualisation of architecture
    dummy_input = torch.randn(1, 3, 3, 42, 42)

    output = model(dummy_input)

    graph = make_dot(output, params=dict(model.named_parameters()))

    # Save or render the graph
    graph.render("cnn_graph", format="png")  # Save as PNG

    ## TASK 8: Redefine the criterion to be softmax cross entropy
    #criterion = nn.CrossEntropyLoss()
    criterion = nn.BCEWithLogitsLoss()

    ## TASK 11: Define the optimizer
    # optimizer = None #torch.optim.SGD(model.parameters(), lr=args.learning_rate, momentum=args.sgd_momentum)
    optimizer = torch.optim.SGD(model.parameters(), lr=args.learning_rate, momentum=args.sgd_momentum)

    log_dir = get_summary_writer_log_dir(args)
    print(f"Writing logs to {log_dir}")
    summary_writer = SummaryWriter(
            str(log_dir),
            flush_secs=5
    )
    trainer = Trainer(
        model, train_loader, test_loader, criterion, optimizer, summary_writer, DEVICE
    )

    print("About to train...")
    trainer.train(
        args.epochs,
        args.val_frequency,
        print_frequency=args.print_frequency,
        log_frequency=args.log_frequency,
    )
    print("Finished training.")
    summary_writer.close()


    print("\nTESTING MODEL")
    test_auc = test_model(model, testingdata, test_ground_truth_dict, DEVICE)
    print("AUC:", test_auc, "on test dataset\n")


class CNN(nn.Module):
    def __init__(self, height: int, width: int, channels: int, class_count: int):
        super().__init__()
        self.input_shape = ImageShape(height=height, width=width, channels=channels)
        self.class_count = class_count

        # Define 1st Conv, Pool, and BatchNorm
        
        # Conv 1
        self.conv1 = nn.Conv2d(
            in_channels=self.input_shape.channels,
            out_channels=96,
            kernel_size=(7, 7),
            padding= (0,0), #(2, 2),
        )
        self.initialise_layer(self.conv1)
        
        # Pool 1
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

        # BatchNorm 1
        self.batchnorm1 = nn.BatchNorm2d(self.conv1.out_channels)

        # Define 2nd Conv, Pool, and BatchNorm
        
        # Conv 2
        self.conv2 = nn.Conv2d(
            in_channels=96,
            out_channels=160,
            kernel_size=(3, 3),
            padding= (0,0) #(2, 2)
        )

        self.initialise_layer(self.conv2)
        
        # Pool 2
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

        # BatchNorm 2
        self.batchnorm2 = nn.BatchNorm2d(num_features=self.conv2.out_channels)
        
        # Define 3rd Conv, Pool, and BatchNorm
        
        # Conv 3
        self.conv3 = nn.Conv2d(
            in_channels=160,
            out_channels=288,
            kernel_size=(3, 3),
            padding= (0,0) #(2, 2)
        )

        self.initialise_layer(self.conv3)
        
        # Pool 3
        self.pool3 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

        # BatchNorm 3
        self.batchnorm3 = nn.BatchNorm2d(num_features=self.conv3.out_channels)

        
        # Define FC 1

        self.fc1 = nn.Linear(2592, 512)
        self.initialise_layer(self.fc1)

        # BatchNorm layer after first FC layer
        # import pdb; pdb.set_trace()
        self.batchnorm4 = nn.BatchNorm1d(num_features=self.fc1.out_features)

        
        # Define FC 2

        self.fc2 = nn.Linear(1536, 512)
        self.initialise_layer(self.fc2)

        # BatchNorm layer after second FC layer

        self.batchnorm5 = nn.BatchNorm1d(num_features=self.fc2.out_features)
        
        # Define FC 3

        self.fc3 = nn.Linear(512, 1)
        self.initialise_layer(self.fc3)

        # BatchNorm layer after third FC layer
        self.batchnorm6 = nn.BatchNorm1d(num_features=self.fc3.out_features)
    
    # FORWARD METHOD for running 3 parallel CNNs
    def forward(self, images: torch.Tensor) -> torch.Tensor:
        #images = images.reshape(-1, 3, 42, 42)
        images1 = images[:, 0, :, :, :]
        
        #pass first resolutions through the network
        x1 = F.relu(self.conv1(images1))
        x1 = self.pool1(x1)
        # x1 = self.batchnorm1(x1)

        x1 = F.relu(self.conv2(x1))
        x1 = self.pool2(x1)
        # x1 = self.batchnorm2(x1)
        
        x1 = F.relu(self.conv3(x1))
        x1 = self.pool3(x1)
        # x1 = self.batchnorm3(x1)

        # Flatten the output of the pooling layer so it is of shape (batch_size, 4096)
        x1 = torch.flatten(x1, start_dim=1)
        
        # Pass x through the first fully connected layer
        x1 = F.relu(self.fc1(x1))
        # x1 = self.batchnorm4(x1)
        
        images2 = images[:, 1, :, :, :]
        
        #pass second resolutions through the network
        x2 = F.relu(self.conv1(images2))
        x2 = self.pool1(x2)
        # x2 = self.batchnorm1(x2)

        x2 = F.relu(self.conv2(x2))
        x2 = self.pool2(x2)
        # x2 = self.batchnorm2(x2)
        
        x2 = F.relu(self.conv3(x2))
        x2 = self.pool3(x2)
        # x2 = self.batchnorm3(x2)

        # Flatten the output of the pooling layer so it is of shape (batch_size, 4096)
        x2 = torch.flatten(x2, start_dim=1)
        
        # Pass x through the first fully connected layer
        x2 = F.relu(self.fc1(x2))
        # x2 = self.batchnorm4(x2)
        
        
        images3 = images[:, 2, :, :, :]
        
        #pass third resolutions through the network
        x3 = F.relu(self.conv1(images3))
        x3 = self.pool1(x3)
        # x3 = self.batchnorm1(x3)

        x3 = F.relu(self.conv2(x3))
        x3 = self.pool2(x3)
        # x3 = self.batchnorm2(x3)
        
        x3 = F.relu(self.conv3(x3))
        x3 = self.pool3(x3)
        # x3 = self.batchnorm3(x3)

        # Flatten the output of the pooling layer so it is of shape (batch_size, 4096)
        x3 = torch.flatten(x3, start_dim=1)
        
        # Pass x through the first fully connected layer
        x3 = F.relu(self.fc1(x3))
        # x3 = self.batchnorm4(x3)
        
        
        ## TASK 6-2: Pass x through the last fully connected layer
        #IMPORTANT: concatenate fc layers before next steps
        xCat = torch.cat((x1, x2, x3), dim=1)
        x = self.fc2(xCat)
        # x = self.batchnorm5(x)
        
        x = self.fc3(x)
        #x = self.batchnorm6(x)

        return x

    @staticmethod
    def initialise_layer(layer):
        if hasattr(layer, "bias"):
            nn.init.zeros_(layer.bias)
        if hasattr(layer, "weight"):
            nn.init.kaiming_normal_(layer.weight)


class Trainer:
    def __init__(
        self,
        model: nn.Module,
        train_loader: DataLoader,
        val_loader: DataLoader,
        criterion: nn.Module,
        optimizer: Optimizer,
        summary_writer: SummaryWriter,
        device: torch.device,
    ):
        self.model = model.to(device)
        self.device = device
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.criterion = criterion
        self.optimizer = optimizer
        self.summary_writer = summary_writer
        self.step = 0

    def train(
        self,
        epochs: int,
        val_frequency: int,
        print_frequency: int = 20,
        log_frequency: int = 5,
        start_epoch: int = 0
    ):
        self.model.train()
        

        
        #import sys; sys.exit(1)
        for epoch in range(start_epoch, epochs):
            self.model.train()
            
            data_load_start_time = time.time()
             #stuck after this
            for batch, labels in self.train_loader:
                
                batch = batch.to(self.device)
                labels = labels.to(self.device)
                data_load_end_time = time.time()

                # print(type(batch))
                # print(batch.shape)
                # print(type(labels))
                # print(labels[0])
                # import sys; sys.exit(1)


                ## TASK 1: Compute the forward pass of the model, print the output shape
                ##         and quit the program
                logits = self.model.forward(batch).squeeze()
                #print(logits.shape)
                                        
                #import sys; sys.exit(1)

                ## TASK 7: Rename `output` to `logits`, remove the output shape printing
                ##         and get rid of the `import sys; sys.exit(1)`

                # Task 7 done above

                ## TASK 9: Compute the loss using self.criterion and
                ##         store it in a variable called `loss`

                loss = self.criterion(logits, labels.float())

                ## TASK 10: Compute the backward pass

                loss.backward()

                ## TASK 12: Step the optimizer and then zero out the gradient buffers.
                
                #self.optimizer.step()

                #self.optimizer.zero_grad()

                with torch.no_grad():
                    preds = (logits > 0.5).float()
                    accuracy = compute_accuracy(labels, preds)
                    
                data_load_time = data_load_end_time - data_load_start_time
                step_time = time.time() - data_load_end_time
                if ((self.step + 1) % log_frequency) == 0:
                    self.log_metrics(epoch, accuracy, loss, data_load_time, step_time)
                if ((self.step + 1) % print_frequency) == 0:
                    self.print_metrics(epoch, accuracy, loss, data_load_time, step_time)

                self.step += 1
                data_load_start_time = time.time()

            self.summary_writer.add_scalar("epoch", epoch, self.step)
            # if ((epoch + 1) % val_frequency) == 0:
            # HARDCODED VAL FREQUENCY FOR DEBUGGING
            if ((epoch + 1) % 19) == 0:

                self.validate()
                # import sys; sys.exit(1)
                # self.validate() will put the model in validation mode,
                # so we have to switch back to train mode afterwards
                self.model.train()
        

    def print_metrics(self, epoch, accuracy, loss, data_load_time, step_time):
        epoch_step = self.step % len(self.train_loader)
        print(
                f"epoch: [{epoch}], "
                f"step: [{epoch_step}/{len(self.train_loader)}], "
                f"batch loss: {loss:.5f}, "
                f"batch accuracy: {accuracy * 100:2.2f}, "
                f"data load time: "
                f"{data_load_time:.5f}, "
                f"step time: {step_time:.5f}"
        )

    def log_metrics(self, epoch, accuracy, loss, data_load_time, step_time):
        self.summary_writer.add_scalar("epoch", epoch, self.step)
        self.summary_writer.add_scalars(
                "accuracy",
                {"train": accuracy},
                self.step
        )
        self.summary_writer.add_scalars(
                "loss",
                {"train": float(loss.item())},
                self.step
        )
        self.summary_writer.add_scalar(
                "time/data", data_load_time, self.step
        )
        self.summary_writer.add_scalar(
                "time/data", step_time, self.step
        )

    def validate(self):

        print("\nValidating model...")

        auc = test_model(self.model, valdata, val_ground_truth_dict, self.device)

        ##############################
        
        # self.model.eval()

        # dataset_size = len(valdata.dataset)

        # print("\nValidating model...")

        # batch_size = 50
        # num_batches = int(2500 / batch_size)



        # preds_dict = {}

        # for i in range (dataset_size):

        #     # potential bug: getting height and width mixed up, so if results look bad, make sure to check this is right

        #     formatted_filename = format_file_name_fixMap_jpg(valdata.dataset[i]['file'])
        #     height = valdata.dataset[i]['X'].shape[1]
        #     width = valdata.dataset[i]['X'].shape[2]

        #     rescale_to_img = transforms.Resize((height, width))

        #     img_crops = torch.stack([valdata.__getitem__(i * 2500 + j)[0] for j in range(2500)])

        #     preds_rows = []

        #     for j in range(num_batches):
                
        #         batch = img_crops[j * batch_size:(j * batch_size) + batch_size,:,:,:,:]

        #         batch = batch.to(self.device)

        #         preds = self.model(batch).detach().cpu().reshape(1, 50)
        #         preds = (preds > 0.5).float()
        #         preds_rows.append(preds)


        #     sal_map = torch.cat(preds_rows, 0) * 255
        #     # visualise_sal_map(sal_map)

        #     # print(sal_map.shape)

        #     # print(torch.count_nonzero(sal_map[0]))

        #     sal_map = rescale_to_img(sal_map.unsqueeze(0).unsqueeze(0)).squeeze(0).squeeze(0)    # unsqueezing and squeezing done because transforms.Resize requires (., ., H, W) shape tensor

        #     # print(height, width)
        #     # print(sal_map.shape)
            
        #     preds_dict[formatted_filename] = sal_map

        # # print(preds_dict)

        # # MOVE THIS TO OUTER CODE SO THAT WE DON'T CALCULATE THIS DICTIONARY EVERY TIME SINCE WE SIMPLY DON'T NEED TO DO IT EVERY TIME
        # val_ground_truth_dict = get_ground_truth_dict(valdata.dataset)

        # # print(preds_dict.keys() == val_ground_truth_dict.keys())

        # auc = calculate_auc(preds_dict, val_ground_truth_dict)

        ######################

        print("AUC:", auc, "on validation dataset\n")

        return

        # print("OK")
        # import sys; sys.exit(1)

        # results = {"preds": [], "labels": []}
        # total_loss = 0
        # self.model.eval()

        # # No need to track gradients for validation, we're not optimizing.
        # with torch.no_grad():
        #     for batch, labels in self.val_loader:
        #         batch = batch.to(self.device)
        #         labels = labels.to(self.device)
        #         logits = self.model(batch)
        #         loss = self.criterion(logits, labels)
        #         total_loss += loss.item()
        #         preds = logits.argmax(dim=-1).cpu().numpy()
        #         results["preds"].extend(list(preds))
        #         results["labels"].extend(list(labels.cpu().numpy()))

        # accuracy = compute_accuracy(
        #     np.array(results["labels"]), np.array(results["preds"])
        # )
        # average_loss = total_loss / len(self.val_loader)

        # self.summary_writer.add_scalars(
        #         "accuracy",
        #         {"test": accuracy},
        #         self.step
        # )
        # self.summary_writer.add_scalars(
        #         "loss",
        #         {"test": average_loss},
        #         self.step
        # )
        # print(f"validation loss: {average_loss:.5f}, accuracy: {accuracy * 100:2.2f}")


def test_model(model, dataMIT, ground_truth_dict, device):
    
    model.eval()

    dataset_size = len(dataMIT.dataset)

    batch_size = 50
    num_batches = int(2500 / batch_size)

    preds_dict = {}

    for i in range (dataset_size):

        # potential bug: getting height and width mixed up, so if results look bad, make sure to check this is right

        formatted_filename = format_file_name_fixMap_jpg(dataMIT.dataset[i]['file'])
        height = dataMIT.dataset[i]['X'].shape[1]
        width = dataMIT.dataset[i]['X'].shape[2]

        rescale_to_img = transforms.Resize((height, width))

        img_crops = torch.stack([dataMIT.__getitem__(i * 2500 + j)[0] for j in range(2500)])

        preds_rows = []

        for j in range(num_batches):
            
            batch = img_crops[j * batch_size:(j * batch_size) + batch_size,:,:,:,:]

            batch = batch.to(device)

            preds = model(batch).detach().cpu().reshape(1, 50)
            # preds = (preds > 0.5).float()
            preds_rows.append(preds)


        sal_map = torch.cat(preds_rows, 0) * 255
        # visualise_sal_map(sal_map)

        sal_map = rescale_to_img(sal_map.unsqueeze(0).unsqueeze(0)).squeeze(0).squeeze(0)    # unsqueezing and squeezing done because transforms.Resize requires (., ., H, W) shape tensor

        preds_dict[formatted_filename] = sal_map

    auc = calculate_auc(preds_dict, ground_truth_dict)

    return auc


def compute_accuracy(
    labels: Union[torch.Tensor, np.ndarray], preds: Union[torch.Tensor, np.ndarray]
) -> float:
    """
    Args:
        labels: ``(batch_size, class_count)`` tensor or array containing example labels
        preds: ``(batch_size, class_count)`` tensor or array containing model prediction
    """
    assert len(labels) == len(preds)
    return float((labels == preds).sum()) / len(labels)


def get_summary_writer_log_dir(args: argparse.Namespace) -> str:
    """Get a unique directory that hasn't been logged to before for use with a TB
    SummaryWriter.

    Args:
        args: CLI Arguments

    Returns:
        Subdirectory of log_dir with unique subdirectory name to prevent multiple runs
        from getting logged to the same TB log directory (which you can't easily
        untangle in TB).
    """
    tb_log_dir_prefix = (
      f"CNN_bn_"
      f"bs={args.batch_size}_"
      f"lr={args.learning_rate}_"
      f"momentum=0.9_" +
      ("hflip_" if args.data_aug_hflip else "") +
      f"run_"
  )
    i = 0
    while i < 1000:
        tb_log_dir = args.log_dir / (tb_log_dir_prefix + str(i))
        if not tb_log_dir.exists():
            return str(tb_log_dir)
        i += 1
    return str(tb_log_dir)


if __name__ == "__main__":
    args, _ = parser.parse_known_args()
    main(args)


Namespace(dataset_root=PosixPath('/home/tm21064/.cache/torch/datasets'), log_dir=PosixPath('logs'), learning_rate=0.0002, batch_size=128, epochs=20, val_frequency=2, log_frequency=10, print_frequency=10, worker_count=0, sgd_momentum=0, data_aug_hflip=False)
Writing logs to logs/CNN_bn_bs=128_lr=0.0002_momentum=0.9_run_11
About to train...
epoch: [0], step: [9/377], batch loss: 0.97803, batch accuracy: 53.91, data load time: 0.00229, step time: 0.04335
epoch: [0], step: [19/377], batch loss: 1.04873, batch accuracy: 46.88, data load time: 0.00175, step time: 0.04659
epoch: [0], step: [29/377], batch loss: 0.96513, batch accuracy: 47.66, data load time: 0.00165, step time: 0.04257
epoch: [0], step: [39/377], batch loss: 0.93674, batch accuracy: 52.34, data load time: 0.00187, step time: 0.04274
epoch: [0], step: [49/377], batch loss: 0.97203, batch accuracy: 56.25, data load time: 0.00176, step time: 0.04268
epoch: [0], step: [59/377], batch loss: 0.92854, batch accuracy: 59.38, data loa